In [57]:
import pandas as pd

In [58]:
df = pd.read_csv('../../data/df_no24.csv')
df.head()

,Unnamed: 0,date,sp500,cpi,nasdaq,gdp,unrate,usdeur,usdgbp
0,0,2014-02-18,1840.760010,235.547,3675.0,17197.738,6.7,0.72993,0.59661
1,1,2014-02-19,1828.750000,235.547,3654.5,17197.738,6.7,0.72969,0.59809
2,2,2014-02-20,1839.780029,235.547,3671.5,17197.738,6.7,0.72664,0.59930
3,3,2014-02-21,1836.250000,235.547,3664.5,17197.738,6.7,0.72831,0.59967
4,4,2014-02-24,1847.609985,235.547,3688.0,17197.738,6.7,0.72880,0.60018


In [59]:
df = df[['date', 'sp500']]
df.head()

,date,sp500
0,2014-02-18,1840.760010
1,2014-02-19,1828.750000
2,2014-02-20,1839.780029
3,2014-02-21,1836.250000
4,2014-02-24,1847.609985


In [60]:
df.fillna(method='ffill', inplace=True, limit=None, axis=None, downcast=None)
df.head(20)

/var/folders/cp/8x9lqtyj66dc8l1337z6dk180000gn/T/ipykernel_8162/428547990.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True, limit=None, axis=None, downcast=None)
/var/folders/cp/8x9lqtyj66dc8l1337z6dk180000gn/T/ipykernel_8162/428547990.py:1: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  df.fillna(method='ffill', inplace=True, limit=None, axis=None, downcast=None)


,date,sp500
0,2014-02-18,1840.760010
1,2014-02-19,1828.750000
2,2014-02-20,1839.780029
3,2014-02-21,1836.250000
4,2014-02-24,1847.609985
5,2014-02-25,1845.119995
6,2014-02-26,1845.160034
7,2014-02-27,1854.290039
8,2014-02-28,1859.449951
9,2014-03-03,1845.729980


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2485 entries, 0 to 2484
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    2485 non-null   datetime64[ns]
 1   sp500   2485 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 39.0 KB


In [46]:
import torch
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer

# Asumiendo que 'df' es tu DataFrame y que 'date' es tu columna de tiempo y 'sp500' es tu columna de objetivo
df['group'] = 0

df['date'] = df['date'].apply(lambda x: x.toordinal())

# Define los parámetros del conjunto de datos
max_encoder_length = 60
max_prediction_length = 20

training_cutoff = df["date"].max() - max_prediction_length

training = TimeSeriesDataSet(
    df[lambda x: x.date <= training_cutoff],
    time_idx="date",
    target="sp500",
    group_ids=["group"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
)

validation = TimeSeriesDataSet.from_dataset(training, df, predict=True, stop_randomization=True)

# Crea los dataloaders para el entrenamiento y la validación
batch_size = 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)

# Define el modelo
tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.03,
    hidden_size=16,
    attention_head_size=1,
    dropout=0.1,
    hidden_continuous_size=8,
    output_size=7,  # 7 quantiles by default
    loss=pytorch_forecasting.metrics.QuantileLoss(),
    log_interval=10,  # log example every 10 batches
    reduce_on_plateau_patience=4,  # reduce learning automatically
)

# Entrena el modelo
trainer = torch.optim.Adam(tft.parameters(), lr=0.03)
max_epochs = 50

trainer.fit(
    tft,
    train_dataloader=train_dataloader,
    val_dataloaders=val_dataloader,
    max_epochs=max_epochs,
)

# Predice con el modelo
predictions = tft.predict(validation)

ValueError: NaTType does not support toordinal